In [14]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import GooglePalm
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from dotenv import load_dotenv
from langchain.agents import AgentExecutor

In [21]:

db_user = "root"
db_password = "admin"
db_host = "localhost" #"DESKTOP-KHE50IO"
db_name = "classicmodels"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [23]:
db.table_info

'\nCREATE TABLE customers (\n\t`customerNumber` INTEGER NOT NULL, \n\t`customerName` VARCHAR(50) NOT NULL, \n\t`contactLastName` VARCHAR(50) NOT NULL, \n\t`contactFirstName` VARCHAR(50) NOT NULL, \n\tphone VARCHAR(50) NOT NULL, \n\t`addressLine1` VARCHAR(50) NOT NULL, \n\t`addressLine2` VARCHAR(50), \n\tcity VARCHAR(50) NOT NULL, \n\tstate VARCHAR(50), \n\t`postalCode` VARCHAR(15), \n\tcountry VARCHAR(50) NOT NULL, \n\t`salesRepEmployeeNumber` INTEGER, \n\t`creditLimit` DECIMAL(10, 2), \n\tPRIMARY KEY (`customerNumber`), \n\tCONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from customers table:\ncustomerNumber\tcustomerName\tcontactLastName\tcontactFirstName\tphone\taddressLine1\taddressLine2\tcity\tstate\tpostalCode\tcountry\tsalesRepEmployeeNumber\tcreditLimit\n103\tAtelier graphique\tSchmitt\tCarine \t40.32.2555\t54, rue Royale\tNone\tNantes\tNone

In [69]:
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
load_dotenv()

os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# llm = GooglePalm(model_name="chat-bison-001",google_api_key=os.getenv("GOOGLE_API_KEY"))
llm = ChatGoogleGenerativeAI(model='models/gemini-pro',google_api_key=os.getenv("GOOGLE_API_KEY"))

In [70]:
toolkit = SQLDatabaseToolkit(db=db,llm=llm)



In [71]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    handle_parsing_errors=True,
    verbose=True
)

d:\NLP_Project\SQL_CHATBOT\venv\lib\site-packages\langchain_community\agent_toolkits\sql\base.py:127: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [72]:

agent_executor.run("Describe the Order related table and how they are related")



> Entering new AgentExecutor chain...
I can use the sql_db_list_tables function to list all tables in the database. I can also use the sql_db_schema function to view the schema of any table in the database.
Action: sql_db_list_tables
Action Input: ""customers, employees, offices, orderdetails, orders, payments, productlines, productsI can now use the sql_db_schema function to view the schema of each Order related table.
Action: sql_db_schema
Action Input: orders, orderdetails, customers
CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAI

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


RuntimeError: generator raised StopIteration

In [35]:
agent_executor.run("How many tables present in database")



> Entering new AgentExecutor chain...
I can call the sql_db_list_tables tool to get the number of tables
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI now know the final answer
Final Answer: 8

> Finished chain.


'8'

In [36]:
agent_executor.run("how many rows in order table")



> Entering new AgentExecutor chain...
I should first check the list of tables to make sure that the table exists
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI see that orders table exist
Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10102	2003-01-10	2003-01-18	200

'3'

In [37]:
agent_executor.run("how many column is foreigkey in payments table")



> Entering new AgentExecutor chain...
I should check the schema of the payments table using sql_db_schema
Action: sql_db_schema
Action Input: payments
CREATE TABLE payments (
	`customerNumber` INTEGER NOT NULL, 
	`checkNumber` VARCHAR(50) NOT NULL, 
	`paymentDate` DATE NOT NULL, 
	amount DECIMAL(10, 2) NOT NULL, 
	PRIMARY KEY (`customerNumber`, `checkNumber`), 
	CONSTRAINT payments_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from payments table:
customerNumber	checkNumber	paymentDate	amount
103	HQ336336	2004-10-19	6066.78
103	JM555205	2003-06-05	14571.44
103	OM314933	2004-12-18	1676.14
*/There is a foreign key column in the payment table which is customerNumber
Final Answer: 1

> Finished chain.


'1'

In [38]:

agent_executor.run("Describe the PurchaseDetails table")



> Entering new AgentExecutor chain...
I should call sql_db_schema to check the schema for PurchaseDetails table
Action: sql_db_schema
Action Input: PurchaseDetailsError: table_names {'PurchaseDetails'} not found in databaseThere is no PurchaseDetails table in the database, so I cannot describe it.

Final Answer: Table does not exist

> Finished chain.


'Table does not exist'

In [68]:
agent_executor.run("Find the top 5 products with the highest total sales revenue")




> Entering new AgentExecutor chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Add an image to use models/gemini-pro-vision, or switch your model to a text model.

In [40]:
agent_executor.run("List top 3  countries with the highest number of orders")



> Entering new AgentExecutor chain...
I will list the tables to retrieve the details
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the orders table as it contains the necessary information. 
Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10102	2003-01-

"[('USA', 112), ('France', 37), ('Spain', 36)]"

In [61]:
from langchain import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
# llm=HuggingFaceHub(repo_id="google/flan-t5-base")
llm = GooglePalm(model_name="text-bison-001",google_api_key=os.getenv("GOOGLE_API_KEY"))

In [62]:
toolkit = SQLDatabaseToolkit(db=db,llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    handle_parsing_errors=True,
    verbose=True
)


d:\NLP_Project\SQL_CHATBOT\venv\lib\site-packages\langchain_community\agent_toolkits\sql\base.py:127: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [63]:
agent_executor.run("Find the top 5 products with the highest total sales revenue")



> Entering new AgentExecutor chain...


AttributeError: module 'google.generativeai' has no attribute 'generate_content'